Imports

In [1]:
import cv2
import numpy as np
import os

Função para normalizar a imagem para 28x28

In [62]:
def ImgNormalizer(img):
    w = len(img[0])
    h = len(img[1])
    DrawX = []
    DrawY = []
    for x in range(w):
        for y in range(h):
            if img[x][y] == 255:
                DrawX.append(x)
                DrawY.append(y)

    # Se não ouver desenho nenhum, retorna imagens completamente vazias
    if not DrawX:
        imgSmall = np.full((28,28), 0, dtype=np.uint8)
        imgSmall2 = np.full((28,28), 0, dtype=np.uint8)
        return imgSmall,imgSmall2
        
    # Pega os valores mínimos e máximos das coordenadas
    MaxXValue = np.max(DrawX)
    MinXValue = np.min(DrawX)
    MaxYValue = np.max(DrawY)
    MinYValue = np.min(DrawY)
    
    # Calcula as amplitudes
    XAmplitude = MaxXValue - MinXValue
    YAmplitude = MaxYValue - MinYValue
    
    if XAmplitude > YAmplitude:
        # Altera as coordenadas para ir de 0 ate a aplitude maxima e centraliza a amplitude menor
        Amp = XAmplitude
        for i in range(len(DrawX)):
            DrawX[i] = DrawX[i] - MinXValue
            DrawY[i] = DrawY[i] - MinYValue + (Amp/2) - (YAmplitude/2)
    else:
        # Altera as coordenadas para ir de 0 ate a aplitude maxima e centraliza a amplitude menor
        Amp = YAmplitude
        for i in range(len(DrawX)):
            DrawX[i] = DrawX[i] - MinXValue + (Amp/2) - (XAmplitude/2)
            DrawY[i] = DrawY[i] - MinYValue
    
    DrawSmall = []
    for i in range(len(DrawX)):
        # Reduz a amplitude para 28x28 com 2 pixels de borda e arredonda os resultados
        newpair = []
        newpair.append(((23/Amp)*DrawX[i]) + 2)
        newpair.append(((23/Amp)*DrawY[i]) + 2) 
        newpairR = np.int32(np.rint(newpair))
        DrawSmall.append(newpairR)
    # Deixa apenas os pares únicos, excluindo os repetidos após o arredondamento
    DrawSmall = np.unique(DrawSmall,axis=0)

    #Desenha a imagem 28x28
    imgSmall = np.full((28,28), 0, dtype=np.uint8)
    imgSmall2 = np.full((28,28), 0, dtype=np.uint8)
    for pair in DrawSmall:
        imgSmall[pair[0]][pair[1]] = 255
        cv2.circle(imgSmall2,(pair[1],pair[0]),1,(255,255,255),-1)

    return imgSmall,imgSmall2

Classe geradora do desenho

In [63]:
class DrawingApp:
    def __init__(self,path,count,DrawingClass):
        # Variáveis iniciais
        self.path = path
        self.isDrawing = False
        self.count = count
        self.DrawingClass = DrawingClass
    
    def drawLine(self,event,x,y,flags,param):
        #Desenha na imagem enquanto o mouse se move após o botão esquerdo do mouse é pressionado
        #Desenha em 2 imagens ao mesmo tempo: Uma para ter uma referencia do desenho e uma para gerar as coordenadas do mouse
        img1 = param[0]
        img2 = param[1]
        if event == cv2.EVENT_LBUTTONDOWN:
            self.isDrawing = True
        elif event == cv2.EVENT_MOUSEMOVE and self.isDrawing:
            cv2.circle(img1,(x,y),5,(255,255,255),-1)
            cv2.circle(img2,(x,y),0,(255,255,255),-1)
        elif event == cv2.EVENT_LBUTTONUP:
            self.isDrawing = False

    def run(self):
        # Checa se a variável de contagem é maior que zero, se não, nem roda o programa
        if self.count < 1:
            raise Exception("Count needs to be bigger than 0")

        # Cria imagens base vazias
        img = np.full((640,640), 0, dtype=np.uint8)
        img2 = np.full((640,640), 0, dtype=np.uint8)

        countImg = np.full((150,400), 0, dtype=np.uint8)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(countImg, str(self.count), (75, 75), font, 2, (255,255,255), 5, cv2.LINE_AA)


        # Cria um vetor com as duas imagens
        images = [img,img2]

        #gera o nome da janela
        windowName = 'drawing app'
        cv2.namedWindow(windowName)
        windowName2 = 'count'
        cv2.namedWindow(windowName2)

        #chama a função de desenho que chama a função drawLine com os eventos do mouse
        cv2.setMouseCallback(windowName,self.drawLine,images)

        while True:
            cv2.imshow(windowName,images[0])
            cv2.imshow(windowName2,countImg)
            wait = cv2.waitKey(1)
            if wait == ord('q'):
                # Sai do Loop da etapa de desenho
                break
            elif wait == ord('r'):
                # Limpa o conteudo da tela para que novas imagens possam ser geradas
                images[0] = np.full((640,640), 0, dtype=np.uint8)
                images[1] = np.full((640,640), 0, dtype=np.uint8)
            elif wait == ord('s'):
                # Gera os nomes da imagens que vão ser geradas
                ImageClass = self.DrawingClass
                imgSaveNameThin = ImageClass+"-"+"Thin"+"-"+str(self.count)+'.jpg'
                imgSaveNameBold = ImageClass+"-"+"Bold"+"-"+str(self.count)+'.jpg'
                
                # Gera os caminhos para salvar as imagens
                DataBankPathThin = os.path.join(self.path,"DataBase\\Thin\\"+ImageClass+"\\"+imgSaveNameThin)
                DataBankPathBold = os.path.join(self.path,"DataBase\\Bold\\"+ImageClass+"\\"+imgSaveNameBold)

                # Gera as imagens na resolução 28x28
                imagesSmall = ImgNormalizer(images[1])

                # Salva as imagens no caminho selecionado
                if not cv2.imwrite(DataBankPathThin, imagesSmall[0]):
                    raise Exception("Could not write the image")
                if not cv2.imwrite(DataBankPathBold, imagesSmall[1]):
                    raise Exception("Could not write the image")

                self.count = self.count + 1

                countImg = np.full((150,400), 0, dtype=np.uint8)
                cv2.putText(countImg, str(self.count), (75, 75), font, 2, (255,255,255), 5, cv2.LINE_AA)
        
        # Fecha a janela e termina processo
        cv2.destroyAllWindows()

### Labels utlizadas:
 - `SetaCima`
 - `SetaBaixo`
 - `SetaDireita`
 - `SetaEsquerda`

Função Chamada para desenhar

In [64]:
def Drawing():
    # Caminho da pasta atual
    path = os.getcwd()
    # Contagem de imagens o programa começa a gerar, DEVE SER MAIOR QUE ZERO
    Contagem = 1
    # Lable na qual a imagem desenhada vai ser salva, Deve ter uma pasta com esse nome no diretório "DataBase"
    LableImage = "Circulo"
    app = DrawingApp(path,Contagem,LableImage)
    app.run()

Execução

In [65]:
if __name__ == '__main__':
    Drawing()